# Lecture 7 - Extended Formulations

##  Header

In [1]:
# Dependencies: Uncomment and run this cell if you don't have these packages 
using JuMP
using Gurobi
using Random
using Combinatorics

## The all-even set polytope

We are going to look at the "all-evens set" polytope; see the slides for more. 

We start with the perfect formulation. 

In [2]:
# The all-evens set polytope is the convex hull of all n-dimensional
# binary vectors whose entries sum to an even number.

n = 22;

Suppose that we want to optimize a linear function over this polytope.

In [3]:
c = rand(-10:10,n);   # Integer-valued objective function

We start with the perfect formulation using $O(2^n)$ constraints in the original space. We need all odd sets.

In [4]:
# Compute all subsets of 1,..., n

all_subsets = collect(powerset([1:n;]));

# For later use, compute all odd cardinality subsets of 1,..., n

odd_subsets = [odd_set for odd_set in all_subsets if mod(length(odd_set),2) == 1];
print(length(odd_subsets))

2097152

Now we add our constraints and solve the model. We add a 30 second time limit.

In [5]:
optimizer = JuMP.optimizer_with_attributes(Gurobi.Optimizer, "TimeLimit" => 30)

model = JuMP.Model(optimizer)

@variable(model, 0 <= x[1:n]<= 1)
@constraint(model, 
    odd_cut[set in odd_subsets], 
    sum(sum(x[i] for i in [1:n;] if i in set)-sum(x[i] for i in [1:n;] if i ∉ set)) <= length(set)-1)

@objective(model, Max, sum(c[i] * x[i] for i in 1:n))

optimize!(model)

Academic license - for non-commercial use only - expires 2025-01-17
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2097152 rows, 22 columns and 46137344 nonzeros
Model fingerprint: 0xe13f54fe
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+01]
Presolve removed 0 rows and 0 columns (presolve time = 30s) ...
Presolve time: 30.45s

Stopped in 0 iterations and 30.45 seconds
Time limit reached

User-callback calls 18, time in user-callback 0.00 sec


The model is slow to even build! Even after the model is built, it is slow to solve.

Now, let us see the extended formulation. 

We need the even indices up to n.

In [6]:
# Even indices

even_nums = [i for i in [0:n;] if mod(i,2)==0];

k = length(even_nums);

Next, we create the model. Recall that we have nk+n+k variables. 

In [7]:
model_EF = JuMP.Model(optimizer)

@variable(model_EF, 0 <= x[1:n]<= 1)
@variable(model_EF, 0 <= xT[1:k,1:n])
@variable(model_EF, 0 <= lambda[1:n])

@constraint(model_EF, convex_sum[i in [1:n;]], sum(xT[t,i] for t in [1:k;]) == x[i])
@constraint(model_EF, polytope[t in [1:k;]], sum(xT[t,i] for i in [1:n;]) == t*lambda[t])
@constraint(model_EF, bounds[t in [1:k;], i in [1:n;]], xT[t,i] <= lambda[t])
@constraint(model_EF, convex_lambda, sum(lambda[i] for i in [1:n;]) == 1)

@objective(model_EF, Max, sum(c[i] * x[i] for i in 1:n))

optimize!(model_EF)

Academic license - for non-commercial use only - expires 2025-01-17
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 299 rows, 308 columns and 1112 nonzeros
Model fingerprint: 0x9bbd0153
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 0 rows and 13 columns
Presolve time: 0.00s
Presolved: 299 rows, 295 columns, 1099 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.4000000e+01   1.800000e+01   0.000000e+00      0s
     105    3.4000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 105 iterations and 0.01 seconds
Optimal objective  3.400000000e+01

User-callback calls 134, time in user-callback 0.00 sec
